### A3.2.2. Vectorization Reports

> *Vectorization reports are compiler diagnostics that explain whether a loop was vectorized, the chosen vectorization factor, or the reason vectorization failed.*

**Explanation:**

When the compiler auto-vectorizes code, **vectorization reports** let you understand what happened and why. They are essential for diagnosing performance — without them, you are guessing whether your hot loops use SIMD.

**Compiler Flags:**

| Compiler | Flag | Output |
|----------|------|--------|
| Clang/LLVM | `-Rpass=loop-vectorize` | Reports successful vectorizations |
| Clang/LLVM | `-Rpass-missed=loop-vectorize` | Reports failed vectorizations with reason |
| Clang/LLVM | `-Rpass-analysis=loop-vectorize` | Detailed cost model analysis |
| GCC | `-fopt-info-vec-optimized` | Reports successful vectorizations |
| GCC | `-fopt-info-vec-missed` | Reports failed vectorizations |

**Common Failure Reasons:**

- **Loop-carried dependency** — iteration N depends on iteration N-1.
- **Non-contiguous memory access** — pointer aliasing or indirect indexing.
- **Function call in loop body** — unless the callee is inlined or has a vector variant.
- **Complex control flow** — multiple exits, exceptions, or unpredictable branches.
- **Insufficient trip count** — loop too short to benefit from SIMD overhead.

**Pragmas for Guidance:**

```c
#pragma clang loop vectorize(enable)         // encourage vectorization
#pragma clang loop vectorize_width(8)        // request VF=8
#pragma clang loop interleave_count(4)       // request IC=4
__attribute__((noinline)) void f();           // prevent inlining
void g() __attribute__((assume_aligned(32))); // promise alignment
```

**Example Report (Clang):**

```
remark: vectorized loop (vectorization width: 8, interleaved count: 2) [-Rpass=loop-vectorize]
remark: loop not vectorized: could not determine number of loop iterations [-Rpass-missed=loop-vectorize]
```

In [ ]:
from dataclasses import dataclass


@dataclass
class VectorizationReport:
    loop_location: str
    vectorized: bool
    vectorization_width: int
    interleave_count: int
    failure_reason: str

    def display(self):
        if self.vectorized:
            print(f"  ✓ {self.loop_location}: vectorized "
                  f"(width: {self.vectorization_width}, interleave: {self.interleave_count})")
        else:
            print(f"  ✗ {self.loop_location}: {self.failure_reason}")


reports = [
    VectorizationReport("add_arrays:5", True, 8, 2, ""),
    VectorizationReport("dot_product:12", True, 4, 1, ""),
    VectorizationReport("linked_list_sum:20", False, 0, 0,
                        "loop not vectorized: value that could not be identified as reduction is used outside the loop"),
    VectorizationReport("indirect_access:30", False, 0, 0,
                        "loop not vectorized: cannot identify array bounds"),
    VectorizationReport("short_loop:40", False, 0, 0,
                        "loop not vectorized: vectorization is not beneficial (trip count < VF)"),
]

vectorized_loops = [
    report for report in reports
    if report.vectorized
]
failed_loops = [
    report for report in reports
    if not report.vectorized
]

print("Vectorization Report Summary:")
print(f"  Total loops analyzed: {len(reports)}")
print(f"  Vectorized: {len(vectorized_loops)}")
print(f"  Failed: {len(failed_loops)}")

print("\nSuccessful:")
for report in vectorized_loops:
    report.display()

print("\nFailed:")
for report in failed_loops:
    report.display()

**References:**

[📘 LLVM Project. *Auto-Vectorization in LLVM.*](https://llvm.org/docs/Vectorizers.html)

[📘 LLVM Project. *Clang Compiler User's Manual — Diagnostics.*](https://clang.llvm.org/docs/UsersManual.html#options-to-emit-optimization-reports)

---

[⬅️ Previous: SIMD Concepts](./01_single_instruction_multiple_data_concepts.ipynb) | [Next: Work Partitioning ➡️](../03_Parallelism/01_work_partitioning.ipynb)